In [ ]:
import glob
import time
import uproot
import tqdm
import pandas as pd
import numpy as np

# file = "root://eos.cms.rcac.purdue.edu//store/data/Run2016B/SingleMuon/NANOAOD/02Apr2020_ver2-v1/20000/014D129C-DD09-A748-BB1C-81184C4A8DDD.root"
prefix = "root://eos.cms.rcac.purdue.edu/"
prefix_mount = "/eos/purdue/"
dataset = "/store/data/Run2016B/SingleMuon/NANOAOD/02Apr2020_ver2-v1"
datasets = [
    # 2016
    "/store/data/Run2016B/SingleMuon/NANOAOD/02Apr2020_ver2-v1",
    "/store/data/Run2016C/SingleMuon/NANOAOD/02Apr2020-v1",
    "/store/data/Run2016D/SingleMuon/NANOAOD/02Apr2020-v1",
    "/store/data/Run2016E/SingleMuon/NANOAOD/02Apr2020-v1",
    "/store/data/Run2016F/SingleMuon/NANOAOD/02Apr2020-v1",
    "/store/data/Run2016G/SingleMuon/NANOAOD/02Apr2020-v1",
    "/store/data/Run2016H/SingleMuon/NANOAOD/02Apr2020-v1",
    #2017
    "/store/data/Run2017B/SingleMuon/NANOAOD/02Apr2020-v1",
    "/store/data/Run2017C/SingleMuon/NANOAOD/02Apr2020-v1",
    "/store/data/Run2017D/SingleMuon/NANOAOD/02Apr2020-v1",
    "/store/data/Run2017E/SingleMuon/NANOAOD/02Apr2020-v1",
    "/store/data/Run2017F/SingleMuon/NANOAOD/02Apr2020-v1",
    #2018
    "/store/data/Run2018A/SingleMuon/NANOAOD/02Apr2020-v1",
    "/store/data/Run2018B/SingleMuon/NANOAOD/02Apr2020-v1",
    "/store/data/Run2018C/SingleMuon/NANOAOD/02Apr2020-v1",
    "/store/data/Run2018D/SingleMuon/NANOAOD/02Apr2020-v1",
]

files = []
for dataset in datasets:
    ds_files = glob.glob(prefix_mount+dataset+"/*/*root")
    print(f"{dataset}: {len(ds_files)}")
    files.extend(ds_files)
files = [f.replace(prefix_mount, prefix) for f in files]
# files = [files[0], files[1]]
print(len(files), "files")
# print(files[0])

In [ ]:
def xrootd_process(file):
    from XRootD import client
    from XRootD.client.flags import OpenFlags
    tick = time.time()
    with client.File() as f:
        status, response = f.open(file, OpenFlags.READ)
        status, data = f.read() # Reads the whole file
        server = f.get_property('DataServer')
    tock = time.time()
    elapsed = tock - tick
    return pd.DataFrame([{"server": server, "time": elapsed}])

In [ ]:
from dask.distributed import Client, as_completed, TimeoutError, KilledWorker
# Measure time for a list of files
def run_benchmark_xrootd(process, files, use_dask=False, client=None):
    
    tick = time.time()

    nevts_total = 0

    xrootd_df = pd.DataFrame(columns=["server", "time"])

    if use_dask:
        if not client:
            raise "Dask client is missing!"
        futures = client.map(process, files)
        results = []
        failed = 0
        for f in futures:
            try:
                results.append(client.gather(f))
            except:
                failed += 1
        results = client.gather(futures)
        xrootd_df = pd.concat(results)
        print(failed, "failed")
    else:
        for file in tqdm.tqdm(files):
            result = xrootd_process(file)
            xrootd_df = pd.concat([xrootd_df, result])
        # print(xrootd_df)

    tock = time.time()
    elapsed = tock - tick

    print(round(elapsed,3), "s")
    xrootd_df.reset_index(inplace=True)
    return xrootd_df


In [ ]:
# Sequential processing
# run_benchmark_xrootd(xrootd_process, files)

In [ ]:
from dask_gateway import Gateway
gateway = Gateway()
cluster = gateway.new_cluster(
    worker_memory = 8,
    queue = "cms-express",
    # reservation = "DASKTEST",
    env={
        "PYTHONPATH": "/depot/cms/private/users/dkondra/af-benchmark",
        "X509_USER_PROXY": "/depot/cms/private/users/dkondra/x509up_u616617"
    }
)
cluster

In [ ]:
# Process via Dask Gateway cluster
df = run_benchmark_xrootd(xrootd_process, files, use_dask=True, client=cluster.get_client())

In [ ]:
cluster.shutdown()

In [ ]:
gateway.list_clusters()

In [ ]:
df

In [ ]:
df.groupby("server").mean().sort_values(by="time", ascending=False).drop("index", axis=1)

In [ ]:
df.groupby("server").count().sort_values(by="time", ascending=False).drop("index", axis=1)

In [ ]:
df.loc[df.server=="eos-a01.cms.rcac.purdue.edu:1103", "time"].plot.hist()